1. Modify the multipath model from the previous homework assignments so that the transmitter is an array of Nt elements and the receiver is an array of Nr elements. Generate a sequence of a few hundred channel matrices for 4 transmit and 4 receive antennas with optimal element spacing (0.4λ) as the transmitter or receiver moves. Compute the capacity for each channel realization based on the water-filling solution for a given “SNR” (PT/σ2 w) and average over the channels.

(a) Plot the average channel capacity as a function of PT/σ2 w over an appropriate range such as 0 to 30 dB. Use a semilogy scale for the vertical axis (capacity) and dB for the horizontal axis (SNR). Include on the same plot the SISO capacity for one transmit and one receive element and the uninformed channel capacity, with a legend or annotation that labels each curve.

(b) What happens to the capacity as the number of multipaths becomes small

2. Change your model to use circular arrays with 10 elements or so spaced a half wavelength apart so that the arrays can control the radiation patterns in all directions. For a propagation environment model with two or three multipaths, plot the eigenpatterns at the transmit or receive side corresponding to the largest two or three singular values of the channel matrix. Relate these patterns to the physical channel. This can be done by overlaying polar plots of the patterns on an x-y plot of the receiver, transmitter, and scatterer locations.